In [2]:
!pip install --upgrade --force-reinstall boto3 botocore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 35.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 1.1 MB/s eta 0:00:00:00:01
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.14
    Uninstalling urllib3-1.26.14:
      Successfully uninstalled urllib3-1.26.14
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0.1
    Uninstalling jmespath-1.0.1:
      Successfully uninstalled jmespath-1.0.1
  Attempting uninstall: python

In [3]:
!pip install yt_dlp

  Using cached yt_dlp-2023.11.16-py2.py3-none-any.whl (3.1 MB)
  Using cached mutagen-1.47.0-py3-none-any.whl (194 kB)
  Using cached websockets-12.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached pycryptodomex-3.19.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached Brotli-1.1.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (2.8 MB)
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor

import boto3
import botocore
# from boto3.dynamodb.conditions import Attr
from yt_dlp import YoutubeDL

dataset_bucket_name = 'up-scaling-prod-dataset-videos'
dataset_dynamodb_name = 'up_scaling_prod_dataset'

s3 = boto3.client('s3', config=botocore.client.Config(max_pool_connections=500))
dynamodb = boto3.resource('dynamodb')

table = dynamodb.Table(dataset_dynamodb_name)

default_dir = 'dataset_splited'

def download_video(video_id: str, video_url: str, quality_p=1080):
    path = f'./{default_dir}/{video_id}/video_{quality_p}p.webm'
    # Configuration for youtube-dl
    ydl_opts = {
        'format': f'bestvideo[height<={quality_p}]+bestaudio/best[height<={quality_p}]',
        'outtmpl': f'./{default_dir}/%(id)s/video_{quality_p}p.%(ext)s',
    }
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    return path


def upload_file_to_s3(bucket_name: str, key: str, file_path: str):
    s3.upload_file(file_path, bucket_name, key)


def set_dynamo_db_flags(video_id: str):
    # Update the 'upload' attribute to True
    table.update_item(
        Key={'id': video_id},
        UpdateExpression='SET upload = :val',
        ExpressionAttributeValues={
            ':val': True
        }
    )


def youtube_video_to_s3(video_id: str, video_url: str, quality_p):
    video_path = download_video(video_id, video_url, quality_p=quality_p)
    frames_directory = split_video_into_frames(video_path, video_id, quality_p)

    # Remove S3 upload to test SageMaker local dir
    max_workers = 500
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.submit(upload_file_to_s3, dataset_bucket_name, f'{video_id}/video/{quality_p}.webm', video_path)

        # Upload each frame to S3
        for frame_file in os.listdir(frames_directory):
            frame_path = os.path.join(frames_directory, frame_file)
            frame_key = f'{video_id}/frames/{quality_p}/{frame_file}'
            executor.submit(upload_file_to_s3, dataset_bucket_name, frame_key, frame_path)

    set_dynamo_db_flags(video_id)


def split_video_into_frames(video_path: str, video_id: str, quality_p: int):
    frames_directory = f'./{default_dir}/{video_id}/frames_{quality_p}p'
    os.makedirs(frames_directory, exist_ok=True)
    frames_path = os.path.join(frames_directory, 'frame-%04d.png')

    # Run FFmpeg command to extract 5 frames per second
    subprocess.run(['ffmpeg', '-i', video_path, '-r', '1', frames_path], check=True)
    return frames_directory


def handler(event, context):
    response = {
        'Items': [
            {'id': 'tx2DaIcoNLQ'},
            {'id': 'E895PNqSgEI'},
            {'id': 'RW94mABPEpw'},
            {'id': 'Z3nOh8Yig0o'},
            {'id': 'XniKiPMM24g'},
            {'id': 'dWB3iP3ai-o'},
            {'id': 'MoxFkJlVZlA'},
            {'id': 'm3GPb-A9CQ4'},
            {'id': 'BWkt79xkd00'},
            {'id': 'yB7-bVpsOic'},
            {'id': 'WkLTWmlTaJM'},
            {'id': 'AaCGGopQOJ0'},
            {'id': 'OiDgFekE2ak'},
        ]
    }

    print(response)

    max_workers = 10
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Iterate over the items returned by the scan
        for item in response['Items']:
            video_id = item['id']
            video_url = f'https://www.youtube.com/watch?v={video_id}'

            executor.submit(youtube_video_to_s3, video_id, video_url, 480)
            executor.submit(youtube_video_to_s3, video_id, video_url, 1080)

    return {
        'statusCode': 200,
        'body': 'Videos processed successfully'
    }


if __name__ == '__main__':
    start_time = time.time()
    handler('', '')
    end_time = time.time()
    print(f"Time taken to upload directory: {end_time - start_time} seconds")


{'Items': [{'id': 'tx2DaIcoNLQ'}, {'id': 'E895PNqSgEI'}, {'id': 'RW94mABPEpw'}, {'id': 'Z3nOh8Yig0o'}, {'id': 'XniKiPMM24g'}, {'id': 'dWB3iP3ai-o'}, {'id': 'MoxFkJlVZlA'}, {'id': 'm3GPb-A9CQ4'}, {'id': 'BWkt79xkd00'}, {'id': 'yB7-bVpsOic'}, {'id': 'WkLTWmlTaJM'}, {'id': 'AaCGGopQOJ0'}, {'id': 'OiDgFekE2ak'}]}
[youtube] Extracting URL: https://www.youtube.com/watch?v=tx2DaIcoNLQ
[youtube] Extracting URL: https://www.youtube.com/watch?v=RW94mABPEpw
[youtube] Extracting URL: https://www.youtube.com/watch?v=tx2DaIcoNLQ
[youtube] Extracting URL: https://www.youtube.com/watch?v=XniKiPMM24g
[youtube] Extracting URL: https://www.youtube.com/watch?v=RW94mABPEpw
[youtube] tx2DaIcoNLQ: Downloading webpage
[youtube] Extracting URL: https://www.youtube.com/watch?v=E895PNqSgEI
[youtube] Extracting URL: https://www.youtube.com/watch?v=E895PNqSgEI
[youtube] RW94mABPEpw: Downloading webpage
[youtube] Extracting URL: https://www.youtube.com/watch?v=XniKiPMM24g
[youtube] tx2DaIcoNLQ: Downloading webpage


ERROR: [youtube] Z3nOh8Yig0o: Video unavailable. The uploader has not made this video available in your country


[youtube] E895PNqSgEI: Downloading m3u8 information
[youtube] RW94mABPEpw: Downloading m3u8 information
[youtube] RW94mABPEpw: Downloading m3u8 information
[youtube] tx2DaIcoNLQ: Downloading android player API JSON
[youtube] Z3nOh8Yig0o: Downloading android player API JSON
[youtube] XniKiPMM24g: Downloading android player API JSON
[youtube] tx2DaIcoNLQ: Downloading m3u8 information
[youtube] E895PNqSgEI: Downloading m3u8 information
[youtube] XniKiPMM24g: Downloading android player API JSON
[youtube] tx2DaIcoNLQ: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=dWB3iP3ai-o
[youtube] dWB3iP3ai-o: Downloading webpage


ERROR: [youtube] Z3nOh8Yig0o: Video unavailable. The uploader has not made this video available in your country


[youtube] XniKiPMM24g: Downloading m3u8 information
[youtube] XniKiPMM24g: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=dWB3iP3ai-o
[youtube] dWB3iP3ai-o: Downloading webpage
[youtube] dWB3iP3ai-o: Downloading ios player API JSON
[info] E895PNqSgEI: Downloading 1 format(s): 248+251
[youtube] dWB3iP3ai-o: Downloading android player API JSON
[info] XniKiPMM24g: Downloading 1 format(s): 244+251
[info] RW94mABPEpw: Downloading 1 format(s): 616+251
[youtube] dWB3iP3ai-o: Downloading m3u8 information
[youtube] dWB3iP3ai-o: Downloading ios player API JSON
[info] E895PNqSgEI: Downloading 1 format(s): 244+251
[info] RW94mABPEpw: Downloading 1 format(s): 244+251
[info] XniKiPMM24g: Downloading 1 format(s): 248+251
[youtube] dWB3iP3ai-o: Downloading android player API JSON
[info] tx2DaIcoNLQ: Downloading 1 format(s): 244+251
[info] tx2DaIcoNLQ: Downloading 1 format(s): 616+251
[youtube] dWB3iP3ai-o: Downloading m3u8 information
[hlsnative] Downloading m3u

...

[Merger] Merging formats into "./dataset_splited/OiDgFekE2ak/video_1080p.webm"


Deleting original file ./dataset_splited/OiDgFekE2ak/video_1080p.f251.webm (pass -k to keep)
Deleting original file ./dataset_splited/OiDgFekE2ak/video_1080p.f616.mp4 (pass -k to keep)


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers344 speed=0.278x    
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --

Time taken to upload directory: 3220.7020611763 seconds


frame= 1044 fps=0.4 q=-0.0 Lsize=N/A time=00:17:24.00 bitrate=N/A dup=0 drop=30202 speed=0.388x    
video:2124271kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown


In [5]:
import os
import shutil

def copy(src_file, dst_file):
    shutil.copy2(src_file, dst_file)

def duplicate_subdir(subsubdir_path, dataset_quality, video_id, quality_p):
    max_workers = 10000
    with ThreadPoolExecutor(max_workers=max_workers) as executor_subdir:
        # Copy all files from the old directory to the new directory
        for file in os.listdir(subsubdir_path):
            src_file = os.path.join(subsubdir_path, file)
            dst_file = os.path.join(dataset_quality, f'{video_id}-{file}')
            executor_subdir.submit(copy, src_file, dst_file)
    print(f'Done for {video_id} : {quality_p}')

def duplicate_data(default_dir):
    base_directory = f'./{default_dir}'
    dataset_directory = './dataset'
    dataset480p = f'{dataset_directory}/480p'
    dataset1080p = f'{dataset_directory}/1080p'

    # Create dataset directory if it doesn't exist
    if not os.path.exists(dataset_directory):
        os.makedirs(dataset_directory)
    if not os.path.exists(dataset480p):
        os.makedirs(dataset480p)
    if not os.path.exists(dataset1080p):
        os.makedirs(dataset1080p)

    # Loop through each subdirectory in the base directory
    for video_id in os.listdir(base_directory):
        subdir_path = os.path.join(base_directory, video_id)
        if not os.path.isdir(subdir_path):
            continue  # Skip if it's not a directory
        # print(subdir)

        max_workers = 5
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for subsubdir in os.listdir(subdir_path):
                subsubdir_path = os.path.join(subdir_path, subsubdir)
                if not os.path.isdir(subsubdir_path):
                    continue  # Skip if it's not a directory

                # Extract video_id and quality_p from the directory name
                parts = subsubdir_path.split('_')
                if len(parts) < 3:
                    continue  # Skip if the directory name doesn't match the expected pattern

                quality_p = parts[2].replace('p', '')
                dataset_quality = f'{dataset_directory}/{quality_p}p'

                executor.submit(duplicate_subdir, subsubdir_path, dataset_quality, video_id, quality_p)


start_time = time.time()

default_dir = 'dataset_splited'
duplicate_data(default_dir)

end_time = time.time()
print(f"Time taken to upload directory: {end_time - start_time} seconds")

Done for MoxFkJlVZlA : 1080
Done for MoxFkJlVZlA : 480
Done for OiDgFekE2ak : 480
Done for OiDgFekE2ak : 1080
Done for AaCGGopQOJ0 : 480
Done for AaCGGopQOJ0 : 1080
Done for dWB3iP3ai-o : 1080
Done for dWB3iP3ai-o : 480
Done for RW94mABPEpw : 480
Done for RW94mABPEpw : 1080
Done for E895PNqSgEI : 480
Done for E895PNqSgEI : 1080
Done for XniKiPMM24g : 480
Done for XniKiPMM24g : 1080
Done for WkLTWmlTaJM : 480
Done for WkLTWmlTaJM : 1080
Done for tx2DaIcoNLQ : 1080
Done for tx2DaIcoNLQ : 480
Done for yB7-bVpsOic : 1080
Done for yB7-bVpsOic : 480
Done for BWkt79xkd00 : 1080
Done for BWkt79xkd00 : 480
Time taken to upload directory: 177.814594745636 seconds


In [6]:
import os

def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # Skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

# Example usage
directory_path = './dataset/'
size = get_directory_size(directory_path)
print(f"Total size of '{directory_path}' is: {size} bytes")


Total size of './dataset/' is: 13371980100 bytes
